In [1]:
from astropy.coordinates import SkyCoord
import astropy.units as u
from astroquery.vizier import Vizier
import pandas as pd

# Read in the Gaia data
gaia_data = pd.read_csv('GaiaProjectData-result.csv')

# Create a new pandas dataframe to hold the matched ZTF data
ztf_matched_data = pd.DataFrame(columns=['ra', 'dec', 'source_id', 'filter', 'obs_time'])

# Loop over each row of the Gaia data
for index, row in gaia_data.iterrows():
    # Create an astropy SkyCoord object for the Gaia position
    gaia_position = SkyCoord(row['ra'], row['dec'], unit=(u.deg, u.deg), frame='icrs')
    
    # Query the ZTF DR14 catalog for the source within a 5 arcsecond radius of the Gaia position
    ztf_results = Vizier.query_region(gaia_position, radius=5*u.arcsec, catalog='II/397/ztf14')
    
    # Check if the query returned any results
    if len(ztf_results) > 0:
        # Extract the ZTF data from the query results
        ztf_data = ztf_results['II/397/ztf14'].to_pandas()
        
        # Filter the ZTF data for the specified filter (g or r)
        ztf_data_filtered = ztf_data[ztf_data['Fid'] == row['filter']]
        
        # Check if any matches were found
        if len(ztf_data_filtered) > 0:
            # Extract the relevant data for each match and append to the matched data dataframe
            for i in range(len(ztf_data_filtered)):
                obs_time = ztf_data_filtered.iloc[i]['ObsTime']
                source_id = ztf_data_filtered.iloc[i]['SourceID']
                ztf_matched_data = ztf_matched_data.append({'ra': row['ra'], 'dec': row['dec'], 'source_id': source_id, 'filter': row['filter'], 'obs_time': obs_time}, ignore_index=True)


In [2]:
print(ztf_matched_data)
print("done")

Empty DataFrame
Columns: [ra, dec, source_id, filter, obs_time]
Index: []
done
